In [1]:
# imports and settings

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline   
# Pipline allows to chain together multiple data transformers and a final estimator into a single, cohesive object.
from sklearn.ensemble import GradientBoostingRegressor   
# GradientBoostingRegressor is an ensamble ML technique that builds a series of decision trees, each aimed at correcting the errors of the previous ones.
from sklearn.metrics import mean_absolute_error, mean_squared_error 

plt.style.use("seaborn-v0_8") 
pd.set_option("display.max_columns", 50) 

In [7]:
# data loading

data_dir = "../data"

orders = pd.read_csv(f"{data_dir}/olist_orders_dataset.csv",
                    parse_dates = [
                        "order_purchase_timestamp",
                        "order_approved_at",
                        "order_delivered_carrier_date",
                        "order_delivered_customer_date",
                        "order_estimated_delivery_date"
                    ]
)

payments = pd.read_csv(f"{data_dir}/olist_order_payments_dataset.csv")
customers = pd.read_csv(f"{data_dir}/olist_customers_dataset.csv")

order_items = pd.read_csv(f"{data_dir}/olist_order_items_dataset.csv")
products = pd.read_csv(f"{data_dir}/olist_products_dataset.csv")

orders.head()



,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [8]:
# Build order‑level revenue table and attach customers

